- More features (RTP)
- MLP structure
- DAP value function
- Other markets & parameters (transferability)


# **Preparation**

CalcValueNoUnc() and ArbValue()

In [1]:
import scipy.io
import numpy as np
import math
import matplotlib.pyplot as plt
from datetime import date
import tensorflow as tf
from datetime import date
import time
import pandas as pd

In [3]:
# Compute current value function using the value function from the next time period
def CalcValueNoUnc(d, c, P, eta, vi, ed, iC, iD):
    """
    Title: Calculate Risk-Neutral value function using deterministic price
    Inputs:
        d - price right now
        c - marginal discharge cost
        P - power rating w.r.t to energy rating and sampling time,
        i.e., 2hr duration battery with 5min resolution -> P = (1/2)/12 
        eta - efficiency
        vi - input value function for the next time period, which equals to
        v_t(e) where e is sampled from 0 to 1 at the granularity e
        ed - granularity at which vi is sampled, in p.u. to energy rating
    Outputs:
        vo - value function for the current time period sampled at ed
    """
    # add a large number of upper and lower v, where the first point is
    # v_t(0-) = +infty, and the second point is v_t(0), the second largest one is
    # v_t(1), and the largest one is v_t(1+) = -infty
    lNum = 1e5*np.ones((1,))
    v_foo = np.concatenate([lNum, vi, -lNum], axis=0)

    # # calculate soc after charge vC = v_t(e+P*eta)
    vC = v_foo[iC]

    # # calculate soc after discharge vC = v_t(e-P/eta)
    vD = v_foo[iD]

    # calculate CDF and PDF
    FtEC = (vi*eta > d).astype(int) # F_t(v_t(e)*eta)
    FtCC = (vC*eta > d).astype(int) # F_t(v_t(e+P*eta)*eta)
    FtED = ((vi/eta + c)*((vi/eta + c) > 0) > d).astype(int) # F_t(v_t(e)/eta + c) 
    FtDD = ((vD/eta + c)*((vD/eta + c) > 0) > d).astype(int) # F_t(v_t(e-P/eta)/eta + c) 

    # calculate terms
    Term1 = vC * FtCC
    Term2 = d*(vC*eta <= d)*(vi*eta > d)/ eta
    Term3 = vi * (FtED - FtEC)
    Term4 = d*(((vi/eta + c)*((vi/eta + c) > 0)) <= d)*(((vD/eta + c)*((vD/eta + c) > 0))>d) * eta
    Term5 = - c * eta * (FtDD - FtED)
    Term6 = vD * (1-FtDD)

    # output new value function samped at ed
    vo = Term1 + Term2 + Term3 + Term4 + Term5 + Term6
    return vo


def ArbValue(lmp, v, e, P, E, eta, c, N):
    """
        Title: Arbitrage test using value function

        lmp: lambda, electricity price over time period t
        v: price function
        e: SoC
        P: P = Pr * Ts; actual power rating taking time step size into account
        E: 1
        eta: eta = .9; # efficiency
        c: c = 10; # marginal discharge cost - degradation
        N: number of SOC samples, 1001
    """

    iE = np.ceil((N-1)*e/E).astype(int) # find the nearest SoC index. iE here is 1 smaller than MATLAB.

    vF = v.copy() # read the value function
    # charge efficiency: iE+1 to end in Matlab, so iE to end here
    vF[iE+1 :] = vF[iE+1 :] * eta
    # discharge efficiency: 1 to iE-1 in Matlab, so 0 to iE-1 (exclusive) here
    vF[0 : iE] = vF[0 : iE] / eta + c

    # charge index
    if len(np.nonzero(vF >= lmp)[0])>0:
        iC = np.max(np.nonzero(vF >= lmp))
    else:
        iC = None

    # discharge index
    if len(np.nonzero(vF <= lmp)[0])>0:
        iD = np.min(np.nonzero(vF <= lmp))
    else:
        iD = None

    # iF = iC*(iC > iE) + iD*(iD < iE) + iE*(iC <= iE)*(iD >= iE);
    if iC is not None:
        if iC > iE:
            iF = iC
        elif iD is not None:
            if iD < iE:
                iF = iD
            else:
                iF = iE
        else:
            iF = iE
    elif iD is not None:
        if iD < iE:
            iF = iD
        else:
            iF = iE
    else:
        iF = iE

    eF = (iF)/(N-1)*E
    eF = max(min(eF, e + P*eta), e-P/eta)
    pF = (e-eF)/eta*((e-eF) < 0) + (e-eF)*eta*((e-eF) > 0)
    
    return eF, pF

In [4]:
def generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, tlambda):
    '''
    Generate value function v and dowmsampled value function vAvg
    '''

    start_time = time.time()

    # Set final SoC level
    vEnd = np.zeros(Ne)
    vEnd[0:math.floor(ef * 1001)] = 1e2 # Use 100 as the penalty for final discharge level

    # Define the risk-neutral value function and populate the final column.
    # v[0, 0] is the marginal value of 0% SoC at the beginning of day 1, v[Ne, T]is the maringal value of 100% SoC at the beginning of the last operating day
    v = np.zeros((Ne, T+1)) # initialize the value function series
    v[:, -1] = vEnd  # v.shape == (1001, 210241)

    # Process indices: discretize vt by modeling it as an vector v_{t,j} in which each element is associated with equally spaced SoC samples
    es = np.arange(start=0, stop=1+ed, step=ed)

    # the number of samples is J = 1 + E/ed
    Ne = len(es)

    # Calculate soc after charge vC = v_t(e+P*eta)
    eC = es + P*eta  # [0.0375 0.0385 0.0395 ... 1.0355 1.0365 1.0375]
    iC = np.ceil(eC/ed)
    iC[iC > (Ne+1)] = Ne + 1
    iC[iC < 1] = 0
    # print(iC) # [  38.   39.   40. ... 1002. 1002. 1002.]
    # print(iC.shape) # (1001,)


    # Calculate soc after discharge vC = v_t(e-P/eta)
    eD = es - P/eta
    iD = np.floor(eD/ed)
    iD[iD > (Ne+1)] = Ne + 1
    iD[iD < 1] = 0
    # print(iD) # [  0.   0.   0. ... 951. 952. 953.]
    # print(iD.shape) # (1001,)


    # Populate value function
    for t in reversed(range(0, T)): # start from the last day and move backwards
        vi = v[:, t+1] # input value function of next time stamp
        vo = CalcValueNoUnc(tlambda[int(t+24/Ts)], c, P, eta, vi, ed, iC.astype(int), iD.astype(int))
        v[:,t] = vo # record the result
    # print(v)
    # print(v.shape) # (1001, 210241)
    # print(np.sum(v)) # 6210425677.739915, MATLAB: 6.2082e+09

    end_time = time.time()
    print('Time:', end_time - start_time)

    # Downsample: https://stackoverflow.com/questions/14916545/numpy-rebinning-a-2d-array
    vAvg = v[0:1000, :].reshape([num_segment, int(1000/num_segment), v.shape[1], 1]).mean(3).mean(1)

    return v, vAvg

In [5]:
def generate_train(T, DAP, tlambda, start, stop, lastDay, num_DAP, num_RTP, vAvg):
    X_train = np.zeros((T, num_DAP+num_RTP))

    # if num_DAP > 0:
    # DAP: previous (num_DAP-1) prices + current price
    DAP_sub = DAP[::12] # Subsample DAP
    lambda_DA_sub = DAP_sub[:, (len(DAP_sub[0])-lastDay+start-2):(len(DAP_sub[0])-lastDay+stop+1)]
    tlambda_DA_sub = lambda_DA_sub.flatten('F')
    for t in range(T):
        X_train[t, 0:num_DAP] = tlambda_DA_sub[int(t/12)+37-num_DAP : int(t/12)+37]

    # RTP: Previous (num_RTP-1) prices + current price
    for t in range(T):
        X_train[t, num_DAP:num_DAP+num_RTP] = tlambda[t+289-num_RTP : t+289]


    y_train = vAvg.T[0:T, :]
    
    print(X_train.shape)
    print(y_train.shape)

    return X_train, y_train

In [6]:
def train(X_train, y_train, input_size=60, dense_size=60, output_size=50, activation_fn='relu', epochs=10):
    model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_size,)),
        tf.keras.layers.Dense(dense_size, activation=activation_fn),
        tf.keras.layers.Dense(dense_size, activation=activation_fn),
        tf.keras.layers.Dense(output_size)
    ])
    print(model.summary())

    # Since we are doing regression, we use mean squared error here
    model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanSquaredError())
    model.fit(X_train, y_train, epochs=epochs)
    return model

In [7]:
def evaluate_using_train():
    print("="*30)
    print('Evaluating using X_train')
    print("="*30)
    v2 = model.predict(X_train).T
    # print(v2.shape) # (20, 210240)
    tlambda = RTP[:, (len(RTP[0])-lastDay+start-2):(len(RTP[0])-lastDay+stop+1)] # (288, 731)
    tlambda = tlambda.flatten('F')
    '''
    Plot
    '''
    # plt.plot(v2[25,0:287], linestyle = 'dotted')
    # plt.plot(v[25,0:287], linestyle = 'dotted')
    # plt.show()

    '''
    Perform arbitrage
    '''
    eS = np.zeros(T) # generate the SoC series
    # print(eS.shape)

    pS = np.zeros(T) # generate the power series
    # print(pS.shape) # (210240,)

    e = e0 # initial SoC
    # print(e)

    for t in range(T-1): # start from the first day and move forwards
        vv = v2[:, t+1]
        e, p = ArbValue(tlambda[288+t], vv, e, P, 1, eta, c, v2.shape[0])
        eS[t] = e # record SoC
        pS[t] = p # record Power

    ProfitOut = np.sum(pS * tlambda[288:210528]) - np.sum(c * pS[pS>0])
    Revenue = np.sum(pS * tlambda[288:210528])
    print('ProfitOut:', ProfitOut)
    print('Revenue:', Revenue)

    return(ProfitOut)

In [23]:
def evaluate_using_test(model, DAP, RTP, startTest, stopTest, lastDay, num_DAP, num_RTP):
    print("="*30)
    print('Evaluating using X_test')
    print("="*30)
    # Select dates
    startTest = date.toordinal(date(2019, 1, 1)) + 366 - 1
    stopTest = date.toordinal(date(2019, 12, 31)) + 366 - 2
    T2 = int((stopTest-startTest+1)*24/Ts)

    X_test = np.zeros((T2, num_DAP+num_RTP))

    # DAP
    DAP_sub = DAP[::12]
    lambda_DA_sub = DAP_sub[:, (len(DAP_sub[0])-lastDay+startTest-2):(len(DAP_sub[0])-lastDay+stopTest+1)]
    tlambda_DA_sub = lambda_DA_sub.flatten('F')

    for t in range(T2):
        X_test[t, 0:num_DAP] = tlambda_DA_sub[int(t/12)+37-num_DAP : int(t/12)+37]

    # RTP
    tlambda_RTP_test = RTP[:, (len(RTP[0])-lastDay+startTest-2):(len(RTP[0])-lastDay+stopTest+1)]
    tlambda_RTP_test = tlambda_RTP_test.flatten('F')

    for t in range(T2):
        X_test[t, num_DAP:num_DAP+num_RTP] = tlambda_RTP_test[t+289-num_RTP : t+289]
    
    # print(len(DAP_sub[0])-lastDay+startTest-2)
    # print(len(DAP_sub[0])-lastDay+stopTest+1)
    # print(X_test[0])

    start_time = time.time()
    '''
    Predict
    '''
    v3 = model.predict(X_test)
    v3 = v3.T

    '''
    Perform arbitrage
    '''
    eS_test = np.zeros(T2) # generate the SoC series
    pS_test = np.zeros(T2) # generate the power series
    prS_test= np.zeros(T2+1) # generate the profit series
    total_profit_test = np.zeros(T2)
    e = e0 # initial SoC
    
    for t in range(T2-1): # start from the first day and move forwards
        vv = v3[:, t+1]
        e, p = ArbValue(tlambda_RTP_test[288+t], vv, e, P, 1, eta, c, v3.shape[0])
        eS_test[t] = e # record SoC
        pS_test[t] = p # record Power
        prS_test[t] = np.sum(pS_test[t] * tlambda_RTP_test[288+t]) - np.sum(c * max(0,pS_test[t]))
        total_profit_test[t] = np.sum(prS_test[0:t])
    ProfitOutTest = np.sum(pS_test * tlambda_RTP_test[288:105120]) - np.sum(c * pS_test[pS_test>0])
    RevenueTest = np.sum(pS_test * tlambda_RTP_test[288:105120])
    DischargeTest = np.sum(pS_test[pS_test>0])
    
    end_time = time.time()
    
    A1 = np.reshape(eS_test,len(eS_test)).tolist()
    A2 = np.reshape(pS_test,len(pS_test)).tolist()
    A3 = np.reshape(prS_test,len(prS_test)).tolist()
    A4 = np.reshape(total_profit_test,len(total_profit_test)).tolist()

    df = pd.DataFrame({'Power': A1, 'Energy': A2, 'Profit': A3, 'Total Profit': A4})
    df.to_csv('result.csv')

    print(round(ProfitOutTest))
    print(round(RevenueTest))
    print(round(((pS_test>0)*pS_test).sum(0)))
    print('Time:', end_time - start_time)
    return v3, ProfitOutTest
#     return v3, ProfitOutTest, RevenueTest, DischargeTest

# **Multi Layor Perceptron Regression**

- Train on 2017 & 2018, Test on 2019
- DAP (t-11 to t+12) and RTP (t-11 to t), 36 features in total
- MLP with one hidden layer of 48 neurons, 10 epochs
- X_train result: 35949.447319685394, 43289.25433722925
- X_test result: 8469.596034343187, 10904.413578202835

In [9]:
RTP_NYC = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_NYC_2010_2019.mat')['RTP'])
DAP_NYC = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_NYC_2010_2019.mat')['DAP'])
RTP_LONGIL = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_LONGIL_2010_2019.mat')['RTP'])
DAP_LONGIL = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_LONGIL_2010_2019.mat')['DAP'])
RTP_NORTH = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_NORTH_2010_2019.mat')['RTP'])
DAP_NORTH = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_NORTH_2010_2019.mat')['DAP'])
RTP_WEST = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_WEST_2010_2019.mat')['RTP'])
DAP_WEST = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_WEST_2010_2019.mat')['DAP'])

RTP_WALNUT = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WALNUT.mat')['Q'])
RTP_WESTLNDS = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WESTLNDS.mat')['Q'])
RTP_WHITMORE = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WHITMORE.mat')['Q'])

RTP = RTP_NYC
DAP = DAP_NYC

# RTP = RTP_LONGIL
# DAP = DAP_LONGIL


# RTP = RTP_NORTH
# DAP = DAP_NORTH

# RTP = RTP_WEST
# DAP = DAP_WEST

# RTP = RTP_WALNUT
# DAP = RTP_WALNUT

In [10]:
'''
Select dates: The data ends on 2019/12/31. We take the data range 2017/1/1 to 2018/12/31
'''

Ts = 1/12 # time step: 5min

# Last day for New England
lastDay = date.toordinal(date(2019, 12, 31)) + 366 - 1 # 737789, MATLAB: 737790
# Last day for California
# lastDay = date.toordinal(date(2021, 12, 31)) + 366 - 1 # 737789, MATLAB: 737790

start = date.toordinal(date(2017, 1, 1)) + 366 - 1 # 736695, MATLAB: 736696
stop = date.toordinal(date(2018, 12, 31)) + 366 - 1 # 737424, MATLAB: 737425

T = int((stop-start+1)*24/Ts) # T: 210240, MATLAB: 210240

# tlambda: real time price over time period t
tlambda = RTP[:, (len(RTP[0])-lastDay+start-2):(len(RTP[0])-lastDay+stop+1)] # (288, 731)
tlambda = tlambda.flatten('F')
# tlambda_DA: day ahead price over time period t
tlambda_DA = DAP[:, (len(DAP[0])-lastDay+start-2):(len(DAP[0])-lastDay+stop+1)] # (288, 731)
print(len(DAP[0])-lastDay+start-2)
print(len(DAP[0])-lastDay+stop+1)
print(tlambda_DA.shape)
tlambda_DA = tlambda_DA.flatten('F') # (210528,)


'''
Set parameters
'''
Pr = 0.5  # normalized power rating wrt energy rating (highest power input allowed to flow through particular equipment)
P = Pr*Ts  # actual power rating taking time step size into account, 0.5*1/12 = 0.041666666666666664
eta = 1.0  # efficiency
c = 10  # marginal discharge cost - degradation
ed = .001  # SoC sample granularity
ef = .5  # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
Ne = math.floor(1/ed)+1  # number of SOC samples, (1/0.001)+1=1001
e0 = .5  # Beginning SoC level


'''
Downsample settings
'''
num_segment = 50

2556
3288
(288, 732)


In [11]:
v, vAvg = generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, tlambda)

Time: 29.043949604034424


In [12]:
startTest = date.toordinal(date(2019, 1, 1)) + 366 - 1
stopTest = date.toordinal(date(2019, 12, 31)) + 366 - 2
print(startTest)
print(stopTest)

737425
737788


In [24]:
num_DAP = 24
num_RTP = 36
num_neurons = 60
num_epochs = 10
num_model = 1
profit = np.zeros([num_model,5])

for i in range(num_model):
  tf.keras.utils.set_random_seed(i)
  tf.random.set_seed(i)
  # tf.config.experimental.enable_op_determinism()
  X_train, y_train = generate_train(T, DAP, tlambda, start, stop, lastDay, num_DAP, num_RTP, vAvg)
  model = train(X_train, y_train, num_DAP+num_RTP, num_neurons, num_segment, 'relu', num_epochs)
  model.save('G:\\My Drive\\Energy-Storage-MLP\\DAP24-RTP36\\eta1\\NYC-Model\\model%s'%i)

  train_profit = evaluate_using_train()

  print('NYC:')
  v1_NYC, test_profit_nyc = evaluate_using_test(model, DAP_NYC, RTP_NYC, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nLONGIL:')
#   v1_LONGIL, test_profit_longil = evaluate_using_test(model, DAP_LONGIL, RTP_LONGIL, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nNORTH:')
#   v1_NORTH, test_profit_north = evaluate_using_test(model, DAP_NORTH, RTP_NORTH, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nWEST:')
#   v1_WEST, test_profit_west = evaluate_using_test(model, DAP_WEST, RTP_WEST, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   profit[i,0] = train_profit
#   profit[i,1] = test_profit_nyc
#   profit[i,2] = test_profit_longil
#   profit[i,3] = test_profit_north
#   profit[i,4] = test_profit_west

# profit_df = pd.DataFrame(profit, columns=['Training profit', 'Testing profit NYC', 'Testing profit LONGIL', 'Testing profit NORTH', 'Testing profit WEST'])
# profit_df.to_csv('G:\\My Drive\\Energy-Storage-MLP\\DAP24-RTP36\\NYC-Model\\profit.csv')

(210240, 60)
(210240, 50)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 60)                3660      
                                                                 
 dense_13 (Dense)            (None, 60)                3660      
                                                                 
 dense_14 (Dense)            (None, 50)                3050      
                                                                 
Total params: 10,370
Trainable params: 10,370
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
6570/6570 [==============================] - 4s 636us/step - loss: 237.7460
Epoch 2/10
6570/6570 [==============================] - 4s 669us/step - loss: 201.4572
Epoch 3/10
6570/6570 [==============================] - 4s 631us/step - loss: 196.7918
Epoch 4/10
6570/6570 [

In [ ]:
num_DAP = 0
num_RTP = 36
num_neurons = 60
num_epochs = 10
num_model = 10
profit = np.zeros([num_model,2])

for i in range(num_model):
  tf.keras.utils.set_random_seed(i)
  tf.random.set_seed(i)
  X_train, y_train = generate_train(T, DAP, tlambda, start, stop, lastDay, num_DAP, num_RTP, vAvg)
  model = train(X_train, y_train, num_DAP+num_RTP, num_neurons, num_segment, 'relu', num_epochs)
  model.save('G:\\My Drive\\Energy-Storage-MLP\\DAP0-RTP36\\NYC-Model\\model%s'%i)
  train_profit = evaluate_using_train()

  print('NYC:')
  v2_NYC, test_profit = evaluate_using_test(model, DAP_NYC, RTP_NYC, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nLONGIL:')
#   v2_LONGIL, test_profit = evaluate_using_test(model, DAP_LONGIL, RTP_LONGIL, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nNORTH:')
#   v2_NORTH, test_profit = evaluate_using_test(model, DAP_NORTH, RTP_NORTH, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nWEST:')
#   v2_WEST, test_profit = evaluate_using_test(model, DAP_WEST, RTP_WEST, startTest, stopTest, lastDay, num_DAP, num_RTP)
  profit[i,0] = train_profit
  profit[i,1] = test_profit
profit_df = pd.DataFrame(profit, columns=['Training profit', 'Testing profit'])
profit_df.to_csv('G:\\My Drive\\Energy-Storage-MLP\\DAP0-RTP36\\NYC-Model\\profit.csv')

In [ ]:
num_DAP = 0
num_RTP = 288
num_neurons = 256
num_epochs = 20
num_model = 10
profit = np.zeros([num_model,2])

for i in range(num_model):
  tf.keras.utils.set_random_seed(i)
  tf.random.set_seed(i)
  X_train, y_train = generate_train(T, DAP, tlambda, start, stop, lastDay, num_DAP, num_RTP, vAvg)
  model = train(X_train, y_train, num_DAP+num_RTP, num_neurons, num_segment, 'relu', num_epochs)
  model.save('G:\\My Drive\\Energy-Storage-MLP\\DAP0-RTP288\\NYC-Model\\model%s'%i)

  train_profit = evaluate_using_train()

  print('NYC:')
  v3_NYC, test_profit = evaluate_using_test(model, DAP_NYC, RTP_NYC, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nLONGIL:')
#   v3_LONGIL = evaluate_using_test(model, DAP_LONGIL, RTP_LONGIL, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nNORTH:')
#   v3_NORTH, test_profit  = evaluate_using_test(model, DAP_NORTH, RTP_NORTH, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nWEST:')
#   v3_WEST, test_profit  = evaluate_using_test(model, DAP_WEST, RTP_WEST, startTest, stopTest, lastDay, num_DAP, num_RTP)
  profit[i,0] = train_profit
  profit[i,1] = test_profit
profit_df = pd.DataFrame(profit, columns=['Training profit', 'Testing profit'])
profit_df.to_csv('G:\\My Drive\\Energy-Storage-MLP\\DAP0-RTP288\\NYC-Model\\profit.csv')

In [ ]:
num_DAP = 24
num_RTP = 288
num_neurons = 256
num_epochs = 20
num_model = 10
profit = np.zeros([num_model,2])

for i in range(num_model):
  tf.keras.utils.set_random_seed(i)
  tf.random.set_seed(i)
  X_train, y_train = generate_train(T, DAP, tlambda, start, stop, lastDay, num_DAP, num_RTP, vAvg)
  model = train(X_train, y_train, num_DAP+num_RTP, num_neurons, num_segment, 'relu', num_epochs)
  model.save('G:\\My Drive\\Energy-Storage-MLP\\DAP24-RTP288\\NYC-Model\\model%s'%i)

  train_profit = evaluate_using_train()

  print('NYC:')
  v4_NYC, test_profit = evaluate_using_test(model, DAP_NYC, RTP_NYC, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nLONGIL:')
#   v4_LONGIL, test_profit = evaluate_using_test(model, DAP_LONGIL, RTP_LONGIL, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nNORTH:')
#   v4_NORTH, test_profit = evaluate_using_test(model, DAP_NORTH, RTP_NORTH, startTest, stopTest, lastDay, num_DAP, num_RTP)
#   print('\nWEST:')
#   v4_WEST, test_profit = evaluate_using_test(model, DAP_WEST, RTP_WEST, startTest, stopTest, lastDay, num_DAP, num_RTP)
  profit[i,0] = train_profit
  profit[i,1] = test_profit
profit_df = pd.DataFrame(profit, columns=['Training profit', 'Testing profit'])
profit_df.to_csv('G:\\My Drive\\Energy-Storage-MLP\\DAP24-RTP288\\NYC-Model\\profit.csv')

In [ ]:
# Generate value function for 2019
def generate_value_function_test(DAP, RTP):
    test_start = date.toordinal(date(2019, 1, 1)) + 366 - 1
    test_stop = date.toordinal(date(2019, 12, 31)) + 366 - 1

    T = int((test_stop-test_start+1)*24/Ts)

    testlambda = RTP[:, (len(RTP[0])-lastDay+test_start-2):(len(RTP[0])-lastDay+test_stop+1)]
    testlambda = testlambda.flatten('F')
    testlambda_DA = DAP[:, (len(DAP[0])-lastDay+test_start-2):(len(DAP[0])-lastDay+test_stop+1)]
    testlambda_DA = tlambda_DA.flatten('F') # (210528,)

    Pr = 0.5  # normalized power rating wrt energy rating (highest power input allowed to flow through particular equipment)
    P = Pr*Ts  # actual power rating taking time step size into account, 0.5*1/12 = 0.041666666666666664
    eta = .9  # efficiency
    c = 30  # marginal discharge cost - degradation
    ed = .001  # SoC sample granularity
    ef = .5  # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
    Ne = math.floor(1/ed)+1  # number of SOC samples, (1/0.001)+1=1001
    e0 = .5  # Beginning SoC level

    num_segment = 50

    vEnd = np.zeros(Ne)
    vEnd[0:math.floor(ef * 1001)] = 1e2 # Use 100 as the penalty for final discharge level

    v = np.zeros((Ne, T+1)) # initialize the value function series
    v[:, -1] = vEnd  # v.shape == (1001, 210241)

    es = np.arange(start=0, stop=1+ed, step=ed)

    Ne = len(es)

    eC = es + P*eta
    iC = np.ceil(eC/ed)
    iC[iC > (Ne+1)] = Ne + 1
    iC[iC < 1] = 0

    eD = es - P/eta
    iD = np.floor(eD/ed)
    iD[iD > (Ne+1)] = Ne + 1
    iD[iD < 1] = 0


    for t in reversed(range(0, T)): # start from the last day and move backwards
        vi = v[:, t+1] # input value function of next time stamp
        vo = CalcValueNoUnc(testlambda[int(t+24/Ts)], c, P, eta, vi, ed, iC.astype(int), iD.astype(int))
        v[:,t] = vo # record the result

    vAvg = v[0:1000, :].reshape([num_segment, int(1000/num_segment), v.shape[1], 1]).mean(3).mean(1)
    return vAvg

vAvg_NYC = generate_value_function_test(DAP_NYC, RTP_NYC)
vAvg_LONGIL = generate_value_function_test(DAP_LONGIL, RTP_LONGIL)
vAvg_NORTH = generate_value_function_test(DAP_NORTH, RTP_NORTH)
vAvg_WEST = generate_value_function_test(DAP_WEST, RTP_WEST)

In [ ]:
vAvg_NYC = vAvg_NYC[:,0:104832]
vAvg_LONGIL = vAvg_LONGIL[:,0:104832]
vAvg_NORTH = vAvg_NORTH[:,0:104832]
vAvg_WEST = vAvg_WEST[:,0:104832]

In [ ]:
RTP = RTP_WEST
v3=vAvg_WEST
c = 30
Pr = 0.5
P = Pr*Ts
eta = .9  # efficiency
ed = .001  # SoC sample granularity
ef = .5  # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
Ne = math.floor(1/ed)+1  # number of SOC samples, (1/0.001)+1=1001
e0 = .5  # Beginning SoC level
startTest = date.toordinal(date(2019, 1, 1)) + 366 - 1
stopTest = date.toordinal(date(2019, 12, 31)) + 366 - 2
T2 = int((stopTest-startTest+1)*24/Ts)
tlambda_RTP_test = RTP[:, (len(RTP[0])-lastDay+startTest-2):(len(RTP[0])-lastDay+stopTest+1)]
tlambda_RTP_test = tlambda_RTP_test.flatten('F')
eS_test = np.zeros(T2) # generate the SoC series
pS_test = np.zeros(T2) # generate the power series
e = e0 # initial SoC
for t in range(T2-1): # start from the first day and move forwards
    vv = v3[:, t+1]
    e, p = ArbValue(tlambda_RTP_test[288+t], vv, e, P, 1, eta, c, v3.shape[0])
    eS_test[t] = e # record SoC
    pS_test[t] = p # record Power
ProfitOutTest = np.sum(pS_test * tlambda_RTP_test[288:105120]) - np.sum(c * pS_test[pS_test>0])
ProfitOutTest

In [ ]:
Ts = 1/12 # time step: 5min
num_DAP = 24
num_RTP = 36
num_segment = 50
RTP = RTP_WEST
DAP = DAP_WEST
lastDay = date.toordinal(date(2019, 12, 31)) + 366 - 1 # 737789, MATLAB: 737790
start = date.toordinal(date(2017, 1, 1)) + 366 - 1 # 736695, MATLAB: 736696
stop = date.toordinal(date(2018, 12, 31)) + 366 - 1 # 737424, MATLAB: 737425
startTest = date.toordinal(date(2019, 1, 1)) + 366 - 1
stopTest = date.toordinal(date(2019, 12, 31)) + 366 - 2
# profit = np.zeros([10,2])
c = 50
Pr = 1
P = Pr*Ts
eta = .9  # efficiency
ed = .001  # SoC sample granularity
ef = .5  # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
Ne = math.floor(1/ed)+1  # number of SOC samples, (1/0.001)+1=1001
e0 = .5  # Beginning SoC level
i = 1
# for i in range(10):
# v, vAvg = generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, tlambda)
# tf.keras.utils.set_random_seed(i)
# tf.random.set_seed(i)
# X_train, y_train = generate_train(T, DAP, tlambda, start, stop, lastDay, num_DAP, num_RTP, vAvg)
# model = tf.keras.models.load_model('G:\\My Drive\\Energy-Storage-MLP\\ES-Paras\\WEST-Model\\P0.020833333333333332_c50\\model%s'%i)
# model = tf.keras.models.load_model('G:\\My Drive\\Energy-Storage-MLP\\ES-Paras\\WEST-Model\\P0.041666666666666664_c0\\model%s'%i)
model = tf.keras.models.load_model('G:\\My Drive\\Energy-Storage-MLP\\ES-Paras\\WEST-Model\\P0.08333333333333333_c50\\model%s'%i)
# train_profit = evaluate_using_train()
v1_NYC, test_profit, test_revenue, test_discharge = evaluate_using_test(model, DAP, RTP, startTest, stopTest, lastDay, num_DAP, num_RTP)
  # mse = ((vAvg_NYC - v1_NYC)**2).mean(axis=None)
  # print(mse)
#   profit[i,0] = train_profit
#   profit[i,1] = test_profit
# profit_df = pd.DataFrame(profit, columns=['Training profit', 'Testing profit']) 
# profit_df.to_csv('G:\\My Drive\\Energy-Storage-MLP\\DAP0-RTP288\\LONGIL-Model\\profit2.csv')


In [ ]:
print(test_profit)
print(test_revenue)
print(test_discharge)

# **ES Parameters Sensitivity Analysis**

In [ ]:
RTP_NYC = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_NYC_2010_2019.mat')['RTP'])
DAP_NYC = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_NYC_2010_2019.mat')['DAP'])
RTP_LONGIL = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_LONGIL_2010_2019.mat')['RTP'])
DAP_LONGIL = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_LONGIL_2010_2019.mat')['DAP'])
RTP_NORTH = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_NORTH_2010_2019.mat')['RTP'])
DAP_NORTH = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_NORTH_2010_2019.mat')['DAP'])
RTP_WEST = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_WEST_2010_2019.mat')['RTP'])
DAP_WEST = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_WEST_2010_2019.mat')['DAP'])

RTP_WALNUT = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WALNUT.mat')['Q'])
RTP_WESTLNDS = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WESTLNDS.mat')['Q'])
RTP_WHITMORE = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WHITMORE.mat')['Q'])

RTP = RTP_NYC
DAP = DAP_NYC

# RTP = RTP_LONGIL
# DAP = DAP_LONGIL


# RTP = RTP_NORTH
# DAP = DAP_NORTH

# RTP = RTP_WEST
# DAP = DAP_WEST

# RTP = RTP_WALNUT
# DAP = RTP_WALNUT

In [ ]:
'''
Select dates: The data ends on 2019/12/31. We take the data range 2017/1/1 to 2018/12/31
'''

Ts = 1/12 # time step: 5min

# Last day for New England
lastDay = date.toordinal(date(2019, 12, 31)) + 366 - 1 # 737789, MATLAB: 737790
# Last day for California
# lastDay = date.toordinal(date(2021, 12, 31)) + 366 - 1 # 737789, MATLAB: 737790

start = date.toordinal(date(2017, 1, 1)) + 366 - 1 # 736695, MATLAB: 736696
stop = date.toordinal(date(2018, 12, 31)) + 366 - 1 # 737424, MATLAB: 737425

T = int((stop-start+1)*24/Ts) # T: 210240, MATLAB: 210240

# tlambda: real time price over time period t
tlambda = RTP[:, (len(RTP[0])-lastDay+start-2):(len(RTP[0])-lastDay+stop+1)] # (288, 731)
tlambda = tlambda.flatten('F')
# tlambda_DA: day ahead price over time period t
tlambda_DA = DAP[:, (len(DAP[0])-lastDay+start-2):(len(DAP[0])-lastDay+stop+1)] # (288, 731)
print(len(DAP[0])-lastDay+start-2)
print(len(DAP[0])-lastDay+stop+1)
print(tlambda_DA.shape)
tlambda_DA = tlambda_DA.flatten('F') # (210528,)


'''
Set parameters
'''
Pr = np.array([0.25, 0.5, 1.0])  # normalized power rating wrt energy rating (highest power input allowed to flow through particular equipment)
P_list = Pr*Ts  # actual power rating taking time step size into account, 0.5*1/12 = 0.041666666666666664
eta = .9  # efficiency
c_list = np.array([0, 10, 30, 50])  # marginal discharge cost - degradation
ed = .001  # SoC sample granularity
ef = .5  # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
Ne = math.floor(1/ed)+1  # number of SOC samples, (1/0.001)+1=1001
e0 = .5  # Beginning SoC level


'''
Downsample settings
'''
num_segment = 50

In [ ]:
num_DAP = 24
num_RTP = 36
num_neurons = 60
num_epochs = 10
num_model = 10
startTest = date.toordinal(date(2019, 1, 1)) + 366 - 1
stopTest = date.toordinal(date(2019, 12, 31)) + 366 - 2
profit = np.zeros([num_model,2])

for P in P_list:
    for c in c_list:
        v, vAvg = generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, tlambda)
        for i in range(num_model):
            tf.keras.utils.set_random_seed(i)
            tf.random.set_seed(i)
            # tf.config.experimental.enable_op_determinism()
            X_train, y_train = generate_train(T, DAP, tlambda, start, stop, lastDay, num_DAP, num_RTP, vAvg)
            model = train(X_train, y_train, num_DAP+num_RTP, num_neurons, num_segment, 'relu', num_epochs)
            model.save('G:\\My Drive\\Energy-Storage-MLP\\ES-Paras\\NYC-Model\\P%s_c%s\\model%s'%(P, c, i))

            train_profit = evaluate_using_train()

            v1, test_profit, test_revenue, test_discharge = evaluate_using_test(model, DAP, RTP, startTest, stopTest, lastDay, num_DAP, num_RTP)
          
            profit[i,0] = train_profit
            profit[i,1] = test_profit           
        profit_df = pd.DataFrame(profit, columns=['Training profit', 'Testing profit'])
        profit_df.to_csv('G:\\My Drive\\Energy-Storage-MLP\\ES-Paras\\NYC-Model\\\\P%s_c%s\\profit.csv'%(P, c))


# **Ground Truth**

## Generate Value Function

### Select dates

In [50]:
Ts = 1/12; # time step
DD = 1; # select days to look back

In [51]:
# lambda: Electricity price over time period t. 
# In this case it's today and yesterday, so 576 timestamps in total.

lmbd = RTP[:, len(RTP[0])-DD-1:len(RTP[0])].flatten('F')
lmbd_DA = DAP[:, len(DAP[0])-DD-1:len(DAP[0])].flatten('F')
print(lmbd.shape)
print(lmbd_DA.shape)

(576,)
(576,)


In [52]:
# Number of time steps
T = len(lmbd)
print(T)

576


### Set parameters

In [53]:
Pr = .5; # normalized power rating wrt energy rating (highest power input allowed to flow through particular equipment)
P = Pr*Ts; # actual power rating taking time step size into account
eta = .9; # efficiency
c = 10; # marginal discharge cost - degradation
ed = .001; # SoC sample granularity
ef = .5; # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
Ne = math.floor(1/ed)+1; # number of SOC samples
e0 = .5; # Beginning SoC level?

print(Pr)
print(P)
print(eta)
print(c)
print(ed)
print(ef)
print(Ne)
print(e0)

0.5
0.041666666666666664
0.9
10
0.001
0.5
1001
0.5


### Generate Value Function

In [54]:
# Generate value function samples
vEnd = np.zeros(Ne)
print(vEnd.shape)

# Use 100 as the penalty for final discharge level
vEnd[0:math.floor(ef * 1000)] = 1e2 # vEnd has 100 from position 0 to 499
print(math.floor(ef * 1000))
print(vEnd)

(1001,)
500
[100. 100. 100. ...   0.   0.   0.]


-------

In [55]:
# TODO: tic

# v: the risk-neutral value function
# In this case, v is 1001*577. 1001 values for 577 time stamps
v = np.zeros((Ne, T+1)) # initialize the value function series

# v[0, 0] is the marginal value of 0% SoC at the beginning of day 1
# v[Ne, T]is the marginal value of 100% SoC at the beginning of the last operating day

v[:, -1] = vEnd

print(v)
print(v.shape)

[[  0.   0.   0. ...   0.   0. 100.]
 [  0.   0.   0. ...   0.   0. 100.]
 [  0.   0.   0. ...   0.   0. 100.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
(1001, 577)


In [56]:
# process index
# discretize vt by modeling it as an vector v_{t,j} in which each element is associated with equally spaced SoC samples
es = np.arange(start=0, stop=1+ed, step=ed)

# the number of samples is J = 1 + E/ed
Ne = len(es)

print(es)
print(Ne)

[0.    0.001 0.002 ... 0.998 0.999 1.   ]
1001


In [57]:
# calculate soc after charge vC = v_t(e+P*eta)
eC = es + P*eta; 
print(eC)

# round to the nearest sample 
# 0, 1, 2, ... 1001, 1002
iC = np.ceil(eC/ed)
print(iC)

iC[iC > (Ne+1)] = Ne + 1
print(iC)

iC[iC < 1] = 0
print(iC)

print(iC.shape)

# import matplotlib.pyplot as plt

# plt.plot(iC, linestyle = 'dotted')
# plt.show()

[0.0375 0.0385 0.0395 ... 1.0355 1.0365 1.0375]
[  38.   39.   40. ... 1036. 1037. 1038.]
[  38.   39.   40. ... 1002. 1002. 1002.]
[  38.   39.   40. ... 1002. 1002. 1002.]
(1001,)


In [58]:
# calculate soc after discharge vC = v_t(e-P/eta)
eD = es - P/eta; 
print(eD)

# round to the nearest sample 
# 0, 1, 2, ... 1001, 1002
iD = np.floor(eD/ed)
print(iD)

iD[iD > (Ne+1)] = Ne + 1
print(iD)

iD[iD < 1] = 0
print(iD)

# import matplotlib.pyplot as plt

# plt.plot(iD, linestyle = 'dotted')
# plt.show()

[-0.0462963 -0.0452963 -0.0442963 ...  0.9517037  0.9527037  0.9537037]
[-47. -46. -45. ... 951. 952. 953.]
[-47. -46. -45. ... 951. 952. 953.]
[  0.   0.   0. ... 951. 952. 953.]


In [59]:
for t in reversed(range(0, T)): # start from the last day and move backwards
    vi = v[:, t+1] # input value function from tomorrow
    vo = CalcValueNoUnc(lmbd[t], c, P, eta, vi, ed, iC.astype(int), iD.astype(int))
    v[:,t] = vo # record the result 

print(v)
print(v.shape)
# 1.0805e+07 for MATLAB
print(np.sum(v))

[[ 17.3         17.3         17.3        ... 100.         100.
  100.        ]
 [ 17.3         17.3         17.3        ... 100.         100.
  100.        ]
 [ 17.22222222  17.22222222  17.22222222 ... 100.         100.
  100.        ]
 ...
 [ 11.8         11.8         11.8        ...   0.           0.
    0.        ]
 [ 11.8         11.8         11.8        ...   0.           0.
    0.        ]
 [ 11.8         11.8         11.8        ...   0.           0.
    0.        ]]
(1001, 577)
10805365.297444448


In [ ]:
# TODO: tElasped = toc;

## Perform Arbitrage

In [60]:
eS = np.zeros(T) # generate the SoC series
# print(eS.shape)

pS = np.zeros(T) # generate the power series
# print(eS.shape)

e = e0 # initial SoC
# print(e)


for t in range(T): # start from the first day and move forwards
    vv = v[:, t+1] # read the SoC value for this day
    e, p = ArbValue(lmbd[t], vv, e, P, 1, eta, c, v.shape[0])
    eS[t] = e.copy() # record SoC
    pS[t] = p.copy() # record Power

### Results

In [61]:
# MATLAB: Profit=6.741478e+01, revenue=7.966044e+01>> 
ProfitOut = np.sum(pS * lmbd) - np.sum(c * pS[pS>0])
Revenue = np.sum(pS * lmbd)
print(ProfitOut)
print(Revenue)

67.41477544444442
79.66044211111108


# **Ground Truth (yearly)**

## Generate Value Function

### Select dates

In [113]:
RTP_NYC = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_NYC_2010_2019.mat')['RTP'])
DAP_NYC = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_NYC_2010_2019.mat')['DAP'])
RTP_LONGIL = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_LONGIL_2010_2019.mat')['RTP'])
DAP_LONGIL = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_LONGIL_2010_2019.mat')['DAP'])
RTP_NORTH = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_NORTH_2010_2019.mat')['RTP'])
DAP_NORTH = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_NORTH_2010_2019.mat')['DAP'])
RTP_WEST = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\RTP_WEST_2010_2019.mat')['RTP'])
DAP_WEST = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\DAP_WEST_2010_2019.mat')['DAP'])

RTP_WALNUT = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WALNUT.mat')['Q'])
RTP_WESTLNDS = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WESTLNDS.mat')['Q'])
RTP_WHITMORE = np.array(scipy.io.loadmat('G:\My Drive\Energy-Storage-MLP\WHITMORE.mat')['Q'])


# RTP = RTP_NYC
# DAP = DAP_NYC

# RTP = RTP_LONGIL
# DAP = DAP_LONGIL

# RTP = RTP_NORTH
# DAP = DAP_NORTH

RTP = RTP_WEST
DAP = DAP_WEST

In [64]:
Ts = 1/12; # time step: 5min

In [106]:
lastDay = date.toordinal(date(2019, 12, 31)) + 366 - 1
start = date.toordinal(date(2019, 1, 1)) + 366 - 1
stop = date.toordinal(date(2019, 12, 31)) + 366 - 1

print(lastDay) # MATLAB: 737790
print(start) # MATLAB: 736696
print(stop) # MATLAB: 737425

737789
737425
737789


In [114]:
# tlambda: Electricity price over time period t
# There are 730 days in total, and we flatten the array to 210528
tlambda = RTP[:, (len(RTP[0])-lastDay+start-2):(len(RTP[0])-lastDay+stop)]
print(tlambda.shape)
tlambda = tlambda.flatten('F')                                                                                                                                                                                                                                  
print(tlambda.shape)

(288, 366)
(105408,)


In [108]:
# tlambda: Electricity price over time period t
# There are 730 days in total, and we flatten the array
tlambda_DA = DAP[:, (len(DAP[0])-lastDay+start-2):(len(DAP[0])-lastDay+stop)]
print(tlambda_DA.shape)
tlambda_DA = tlambda_DA.flatten('F')
print(tlambda_DA.shape)

(288, 366)
(105408,)


In [68]:
tlambda_DA

array([24.63, 24.63, 24.63, ..., 17.82, 17.82, 17.82])

In [69]:
# MATLAB: 210240
T = int((stop-start+1)*24/Ts)
print(T)

105120


In [70]:
len(RTP[0])-lastDay+stop

3650

### Set parameters



In [95]:
Pr = .5; # normalized power rating wrt energy rating (highest power input allowed to flow through particular equipment)
P = Pr*Ts; # actual power rating taking time step size into account
eta = 1.0; # efficiency
c = 10; # marginal discharge cost - degradation
ed = .001; # SoC sample granularity
ef = .5; # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
Ne = math.floor(1/ed)+1; # number of SOC samples
e0 = .5; # Beginning SoC level?

### Generate Value Function

In [96]:
# Generate value function samples
vEnd = np.zeros(Ne)
print(vEnd.shape)

# Use 100 as the penalty for final discharge level
vEnd[0:math.floor(ef * 1001)] = 1e2 # vEnd has 100 from position 0 to 9, 0 from 10 to 20
print(math.floor(ef * 1001))
print(vEnd)

(1001,)
500
[100. 100. 100. ...   0.   0.   0.]


In [97]:
# TODO: index debug
# v: the risk-neutral value function
v = np.zeros((Ne, T+1)) # initialize the value function series

# v[0, 0] is the marginal value of 0% SoC at the beginning of day 1
# v[Ne, T]is the maringal value of 100% SoC at the beginning of the last operating day

v[:, -1] = vEnd

print(v)
print(v.shape)

[[  0.   0.   0. ...   0.   0. 100.]
 [  0.   0.   0. ...   0.   0. 100.]
 [  0.   0.   0. ...   0.   0. 100.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
(1001, 105121)


In [98]:
# process index
# discretize vt by modeling it as an vector v_{t,j} in which each element is associated with equally spaced SoC samples
es = np.arange(start=0, stop=1+ed, step=ed)

# the number of samples is J = 1 + E/ed
Ne = len(es)

print(es)
print(Ne)

[0.    0.001 0.002 ... 0.998 0.999 1.   ]
1001


In [99]:
# calculate soc after charge vC = v_t(e+P*eta)
eC = es + P*eta; 
print(eC)

# round to the nearest sample 
# 0, 1, 2, ... 1001, 1002
iC = np.ceil(eC/ed)
print(iC)

iC[iC > (Ne+1)] = Ne + 1
print(iC)

iC[iC < 1] = 0
print(iC)

print(iC.shape)

[0.04166667 0.04266667 0.04366667 ... 1.03966667 1.04066667 1.04166667]
[  42.   43.   44. ... 1040. 1041. 1042.]
[  42.   43.   44. ... 1002. 1002. 1002.]
[  42.   43.   44. ... 1002. 1002. 1002.]
(1001,)


In [100]:
# calculate soc after discharge vC = v_t(e-P/eta)
eD = es - P/eta
print(eD)

# round to the nearest sample 
# 0, 1, 2, ... 1001, 1002
iD = np.floor(eD/ed)
print(iD)

iD[iD > (Ne+1)] = Ne + 1
print(iD)

iD[iD < 1] = 0
print(iD)

print(iD.shape)

[-0.04166667 -0.04066667 -0.03966667 ...  0.95633333  0.95733333
  0.95833333]
[-42. -41. -40. ... 956. 957. 958.]
[-42. -41. -40. ... 956. 957. 958.]
[  0.   0.   0. ... 956. 957. 958.]
(1001,)


In [ ]:
v.shape

In [ ]:
T-1+24/Ts

In [115]:
for t in reversed(range(0, T)): # start from the last day and move backwards
    vi = v[:, t+1] # input value function from tomorrow
    vo = CalcValueNoUnc(tlambda[int(t+24/Ts)], c, P, eta, vi, ed, iC.astype(int), iD.astype(int))
    v[:,t] = vo # record the result 
    # print(tlambda[int(t+24/Ts)])
    # break

print(v)
print(v.shape)
# TODO: MATLAB gives 6.2082e+09. Need to look into further on this.
print(np.sum(v))

[[  5.71   5.71   5.71 ... 100.   100.   100.  ]
 [  5.71   5.71   5.71 ... 100.   100.   100.  ]
 [  5.71   5.71   5.71 ... 100.   100.   100.  ]
 ...
 [  3.09   3.09   3.09 ...   0.     0.     0.  ]
 [  3.09   3.09   3.09 ...   0.     0.     0.  ]
 [  3.09   3.09   3.09 ...   0.     0.     0.  ]]
(1001, 105121)
1826028587.1300006


In [ ]:
tlambda[int(T-2+24/Ts)]

In [ ]:
tlambda[-20:]

### Downscale value function

In [116]:
# bin shape
# a = 50
# b = 1
# vAvg = v[0:1000, :].reshape([1000 // a, a, v.shape[1] // b, b]).mean(3).mean(1)
num_segment = 50
vAvg = v[0:1000, :].reshape([num_segment, int(1000/num_segment), v.shape[1], 1]).mean(3).mean(1)

In [ ]:
vAvg.shape

In [ ]:
vAvg.shape[0]

## Perform Arbitrage

In [117]:
eS = np.zeros(T) # generate the SoC series
# print(eS.shape)

pS = np.zeros(T) # generate the power series
# print(eS.shape)

e = e0 # initial SoC
# print(e)


for t in range(T): # start from the first day and move forwards
    vv = vAvg[:, t+1] # read the SoC value for this day
    e, p = ArbValue(tlambda[t+288], vv, e, P, 1, eta, c, vAvg.shape[0])
    eS[t] = np.float64(e).copy() # record SoC
    pS[t] = np.float64(p).copy() # record Power

### Results

In [118]:
ProfitOut = np.sum(pS * tlambda[288:]) - np.sum(c * pS[pS>0])
Revenue = np.sum(pS * tlambda[288:])
print(ProfitOut)
print(Revenue)

27274.220365646248
34108.31900510203


# Multi Layer Perceptron Regression with Average Prices

In [ ]:
Ts = 1/12; # time step: 5min

lastDay = date.toordinal(date(2019, 12, 31)) + 366 - 1
start = date.toordinal(date(2017, 1, 1)) + 366 - 1
stop = date.toordinal(date(2018, 12, 31)) + 366 - 1


# tlambda: Electricity price over time period t
# There are 731 days in total, and we flatten the array to 210528
# len(RTP[0])-lastDay+start-2 = 2254 (Include one day ahead of start)
# len(RTP[0])-lastDay+stop = 3285 (Include stop)
tlambda = RTP[:, (len(RTP[0])-lastDay+start-2):(len(RTP[0])-lastDay+stop)]
print(tlambda.shape)
tlambda = tlambda.flatten('F')
print(tlambda.shape)


# tlambda: Electricity price over time period t
# There are 731 days in total, and we flatten the array
tlambda_DA = DAP[:, (len(DAP[0])-lastDay+start-2):(len(DAP[0])-lastDay+stop)]
print(tlambda_DA.shape)
tlambda_DA = tlambda_DA.flatten('F')
print(tlambda_DA.shape)


# MATLAB: 210240
T = int((stop-start+1)*24/Ts);
print(T)


Pr = .5; # normalized power rating wrt energy rating (highest power input allowed to flow through particular equipment)
P = Pr*Ts; # actual power rating taking time step size into account
eta = .9; # efficiency
c = 10; # marginal discharge cost - degradation
ed = .001; # SoC sample granularity
ef = .5; # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
Ne = math.floor(1/ed)+1; # number of SOC samples
e0 = .5; # Beginning SoC level?


# Generate value function samples
vEnd = np.zeros(Ne)
print(vEnd.shape)

# Use 100 as the penalty for final discharge level
vEnd[0:math.floor(ef * 1001)] = 1e2 # vEnd has 100 from position 0 to 9, 0 from 10 to 20
print(math.floor(ef * 1001))
print(vEnd)


# TODO: index debug
# v: the risk-neutral value function
v = np.zeros((Ne, T+1)) # initialize the value function series

# v[0, 0] is the marginal value of 0% SoC at the beginning of day 1
# v[Ne, T]is the maringal value of 100% SoC at the beginning of the last operating day

v[:, -1] = vEnd

print(v)
print(v.shape)


# process index
# discretize vt by modeling it as an vector v_{t,j} in which each element is associated with equally spaced SoC samples
es = np.arange(start=0, stop=1+ed, step=ed)

# the number of samples is J = 1 + E/ed
Ne = len(es)

print(es)
print(Ne)


# calculate soc after charge vC = v_t(e+P*eta)
eC = es + P*eta
print(eC)

# round to the nearest sample 
# 0, 1, 2, ... 1001, 1002
iC = np.ceil(eC/ed)
print(iC)

iC[iC > (Ne+1)] = Ne + 1
print(iC)

iC[iC < 1] = 0
print(iC)

print(iC.shape)


# calculate soc after discharge vC = v_t(e-P/eta)
eD = es - P/eta
print(eD)

# round to the nearest sample 
# 0, 1, 2, ... 1001, 1002
iD = np.floor(eD/ed)
print(iD)

iD[iD > (Ne+1)] = Ne + 1
print(iD)

iD[iD < 1] = 0
print(iD)

print(iD.shape)


for t in reversed(range(0, T)): # start from the last day and move backwards
    vi = v[:, t+1] # input value function of next time stamp
    vo = CalcValueNoUnc(tlambda[int(t+24/Ts)], c, P, eta, vi, ed, iC.astype(int), iD.astype(int))
    v[:,t] = vo # record the result


# bin shape
a = 50
b = 1
vAvg = v[0:1000, :].reshape([1000 // a, a, v.shape[1] // b, b]).mean(3).mean(1)

In [ ]:
X_train = np.zeros((T, 24))

# Subsample DAP
DAP_sub = DAP[::12]

# Select dates and flatten
lambda_DA_sub = DAP_sub[:, (len(DAP_sub[0])-lastDay+start-2):(len(DAP_sub[0])-lastDay+stop+1)]
tlambda_DA_sub = lambda_DA_sub.flatten('F')

# Select previous 11 prices and current price
for t in range(T):
    X_train[t, 0:12] = tlambda_DA_sub[int(t/12)+13 : int(t/12)+25]

# Select previous 11 prices and current price
for t in range(T):
    X_train[t, 12:24] = tlambda[t+277 : t+289]


X_train_avg = np.zeros((T,24))

for t in range(T):
    for i in range(12):
        X_train_avg[t, i] = np.mean(X_train[t, i:12])

for t in range(T):
    for i in range(12):
        X_train_avg[t, 12+i] = np.mean(X_train[t, 12+i:24])


y_train = vAvg.T[0:T, :]

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(24,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(20)
])


# Since we are doing regression, we use mean squared error here
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError())


model.fit(X_train_avg, y_train, epochs=10)

In [ ]:
prediction = model.predict( np.array([X_train_avg[11],]))
print(prediction)
plt.plot(prediction[0], linestyle = 'dotted')
plt.show()

In [ ]:
v2 = model.predict(X_train_avg).T
plt.plot(v2[10,0:287], linestyle = 'dotted')
plt.plot(v[10,0:287], linestyle = 'dotted')
plt.show()

In [ ]:
eS = np.zeros(T) # generate the SoC series
# print(eS.shape)

pS = np.zeros(T) # generate the power series
# print(eS.shape)

e = e0 # initial SoC
# print(e)


for t in range(T-1): # start from the first day and move forwards
    vv = v2[:, t+1]
    e, p = ArbValue(tlambda[288+t], vv, e, P, 1, eta, c, v2.shape[0])
    eS[t] = e # record SoC
    pS[t] = p # record Power

In [ ]:
ProfitOut = np.sum(pS * tlambda[288:]) - np.sum(c * pS[pS>0])
Revenue = np.sum(pS * tlambda[288:])
print(ProfitOut)
print(Revenue)

In [ ]:
from datetime import date

startTest = date.toordinal(date(2019, 1, 1)) + 366 - 1
stopTest = date.toordinal(date(2019, 12, 31)) + 366 - 1

T2 = int((stopTest-startTest+1)*24/Ts)


X_test = np.zeros((T2, 24))

tlambda_RTP_test = RTP[:, (len(RTP[0])-lastDay+startTest-2):(len(RTP[0])-lastDay+stopTest)]
print(tlambda_RTP_test.shape)
tlambda_RTP_test = tlambda_RTP_test.flatten('F')
print(tlambda_RTP_test.shape)

for t in range(T2):
    X_test[t, 12:24] = tlambda_RTP_test[t+277 : t+289]

DAP_sub = DAP[::12]

lambda_DA_sub = DAP_sub[:, (len(DAP_sub[0])-lastDay+startTest-2):(len(DAP_sub[0])-lastDay+stopTest+1)]
tlambda_DA_sub = lambda_DA_sub.flatten('F')
tlambda_DA_sub.shape

for t in range(T2):
    X_test[t, 0:12] = tlambda_DA_sub[int(t/12)+13 : int(t/12)+25]


X_test_avg = np.zeros((T2, 24))

for t in range(T2):
    for i in range(12):
        X_test_avg[t, i] = np.mean(X_test[t, i:12])

for t in range(T2):
    for i in range(12):
        X_test_avg[t, 12+i] = np.mean(X_test[t, 12+i:24])

In [ ]:
v3 = model.predict(X_test_avg).T

eS_test = np.zeros(T2) # generate the SoC series
# print(eS.shape)

pS_test = np.zeros(T2) # generate the power series
# print(eS.shape)

e = e0 # initial SoC
# print(e)
 

for t in range(T2-1): # start from the first day and move forwards
    vv = v3[:, t+1]
    e, p = ArbValue(tlambda_RTP_test[288+t], vv, e, P, 1, eta, c, v3.shape[0])
    eS_test[t] = e # record SoC
    pS_test[t] = p # record Power

In [ ]:
ProfitOutTest = np.sum(pS_test * tlambda_RTP_test[288:]) - np.sum(c * pS_test[pS_test>0])
RevenueTest = np.sum(pS_test * tlambda_RTP_test[288:])
print(ProfitOutTest)
print(RevenueTest)